# DenseNet121 - 32x32

### Setup

In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = "3"

In [2]:
import gc
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split

In [3]:
print("CUDA avilable:",
     True if tf.config.list_physical_devices("GPU") else False)

CUDA avilable: True


In [4]:
SEED = 1234
np.random.seed(SEED)
tf.random.set_seed(SEED)

### Preprocessing

In [5]:
PATH = "C:/Users/shj03/Desktop/git_files/ANBW/dataset"
train = pd.read_csv(f"{PATH}/train.csv")
test = pd.read_csv(f"{PATH}/test.csv")
submission = pd.read_csv(f"{PATH}/submission.csv")

In [ ]:
x_train = train.drop(["id", "digit", "letter"], axis=1).values
x_train = x_train.reshape(-1, 28, 28, 1)
x_train = np.where((x_train<=20)&(x_train!=0) ,0.,x_train)
x_train = x_train/255
x_train = x_train.astype("float32")
x_train_resized = np.zeros([2048, 32, 32, 3], dtype=np.float32)

for idx, img in enumerate(x_train):
    converted = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
    resized = cv2.resize(converted, (32, 32), interpolation=cv2.INTER_CUBIC)
    del 